In [22]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import fitz  # PyMuPDF: PDF 텍스트 추출용

# === 1단계: PDF 텍스트 추출 ===
def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

raw_text = extract_text_from_pdf("콘텐츠분쟁해결_사례.pdf")

# === 2단계: 문서 분할 ===
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
split_docs = text_splitter.create_documents([raw_text])

# === 3단계: 임베딩 및 벡터 DB ===
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1536
)
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# === 4단계: LLM 및 QA 체인 ===
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

prompt_template = PromptTemplate(
    template="""
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:""",
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True
)

# === 5단계: 분쟁 유형 분류 함수 ===
def classify_dispute_type(query: str) -> str:
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

# === 6단계: 테스트 실행 ===
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

for question in test_questions:
    dispute_type = classify_dispute_type(question)
    result = qa_chain.invoke({"query": question})
    print(f"\n🟢 질문: {question}")
    print(f"📌 유형: {dispute_type}")
    print(f"🔷 답변: {result['result']}\n")


C:\Users\user\AppData\Local\Temp\ipykernel_30484\2476080031.py:29: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(
c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\langchain-homework-EkbqySkC-py3.12\Lib\site-packages\langchain_community\embeddings\openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_30484\2476080031.py:37: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed i


🟢 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
📌 유형: 게임
🔷 답변: 온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률적 조언을 드리겠습니다. 제시된 사례들을 근거로 하여 설명드리겠습니다.

1. **사례 분석**:
   - **시스템 오류로 인한 아이템 소멸**: 제시된 사례 중 2006년 프로그램 오류로 소멸된 아이템 복구 요구 사례에서는 피신청인이 프로그램 오류를 인정하고 복구를 진행한 바 있습니다. 이는 시스템 오류로 인한 피해가 인정될 경우 복구가 가능하다는 선례를 보여줍니다.
   - **계정 명의 문제**: 첫 번째 사례에서는 계정 명의자가 아닌 경우 복구가 불가하다는 점이 강조되었습니다. 계정 명의가 본인이 아닌 경우, 복구 요청이 거부될 수 있습니다.

2. **관련 법령**:
   - 온라인 디지털콘텐츠산업 발전법 및 전자상거래 등에서의 소비자보호에 관한 법률에 따르면, 온라인 콘텐츠의 이용 조건과 청약철회 제한 사항은 명확히 고지되어야 합니다. 이와 관련하여 게임사의 약관 및 운영정책을 확인하는 것이 중요합니다.

3. **실무적 해결방안**:
   - **계정 명의 확인**: 본인이 계정의 실소유주임을 증명할 수 있는 자료를 준비하여 게임사에 제출합니다. 계정 명의 문제로 복구가 거부된 경우, 명의자와 협력하여 복구 요청을 진행할 수 있습니다.
   - **오류 증명**: 시스템 오류로 인한 아이템 소멸을 증명할 수 있는 자료(스크린샷, 오류 메시지 등)를 수집하여 게임사에 제출합니다.
   - **고객센터 및 분쟁조정 신청**: 게임사의 고객센터에 정식으로 이의를 제기하고, 한국소비자원이나 콘텐츠분쟁조정위원회에 분쟁조정을 신청할 수 있습니다.
   - **약관 검토**: 게임사의 약관을 검토하여 복구 불가 사유가 명시되어 있는지 확인합니다. 약관에 명시되지 않은 경우, 추가적인 법적 조치를 고려할 수